In [ ]:
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

path = Path('/storage/home/adz6/group/project')
result_path = path / 'results' / 'hfss' / 'cavity' / '220314_grooved_cavity_optimization'

def FindTE01p(result_df, design_params, num_modes=10, TE_threshold=10000, num_TE = 2):
    Qs = [f'Q({i+1})' for i in range(num_modes)]
    Modes = [f're(Mode({i+1}))' for i in range(num_modes)]
    TE01p_entries = []
    for i in range(result_df.shape[0]):
        raw_entry = result_df.iloc[i]
        entry_Qs = np.array(raw_entry[Qs].array)
        entry_Modes = np.array(raw_entry[Modes].array)
        TE_inds = np.argwhere(entry_Qs > TE_threshold).squeeze()
        
        for n in range(num_TE):
            _entry = {}
            _entry['mode'] = f'TE01{n+1}'
            try:
                _entry['Q'] = entry_Qs[TE_inds[n]]
                _entry['f'] = entry_Modes[TE_inds[n]]
                if TE_inds[n] < num_modes -1:
                    diff1 = abs(entry_Modes[TE_inds[n]] - entry_Modes[TE_inds[n]-1])
                    diff2 = abs(entry_Modes[TE_inds[n]] - entry_Modes[TE_inds[n]+1])
                    _entry['min_delta_f'] = np.min([diff1, diff2])
                else:
                    diff1 = abs(entry_Modes[TE_inds[n]] - entry_Modes[TE_inds[n]-1])
                    _entry['min_delta_f'] = np.min([diff1])
            except BaseException as err:
                print(err)
                _entry['Q'] = -1
                _entry['f'] = -1
                _entry['min_delta_f'] = -1
                
            for m, param in enumerate(design_params):
                
                if type(raw_entry[param]) == str:
                    _entry[param] = float(raw_entry[param].split('mm')[0])
                else:
                    _entry[param] = raw_entry[param]
                
            TE01p_entries.append(_entry)
            
    return pd.DataFrame.from_records(TE01p_entries)
            
def RankTE011Variations(te01p_df, params):
    
    te011_df = te01p_df[te01p_df['mode']=='TE011']
    te011_df = te011_df[te011_df['Q'] > 0]
    
    for param in params:
        if param == 'min_delta_f':
            norm_param = te011_df[param] / 1e6
        else:
            norm_param = te011_df[param] / te011_df[param].min()
        #print(norm_param)
        te011_df.insert(0, f'norm_{param}', norm_param)
    
    return te011_df
        
                
        
        
    

In [ ]:
result_list = list(result_path.iterdir())
for item in result_list: print(item)

In [ ]:
df_list = []
for item in result_list: df_list.append(pd.read_csv(item))

In [ ]:
design_params = ['Variation', 'groove_depth_fraction', 'groove_width', 'main_cavity_wall_thickness']

te01p_df_list = []
for item in df_list: te01p_df_list.append(FindTE01p(item, design_params, num_modes=16))

In [ ]:
te01p_df_list[0]['main_cavity_wall_thickness'].unique()

# plot Q, f, delta f as a function of design variables

In [ ]:
data_keys = ['Q', 'f', 'min_delta_f']

te011_data_dict = {}
model_labels = [15, 18]

for i in range(len(te01p_df_list)):
    
    df = te01p_df_list[i]
    
    te011_data_dict[model_labels[i]] = {}
    
    depth_array = df['groove_depth_fraction'].unique()
    width_array = df['groove_width'].unique()
    
    depth_grid, width_grid = np.meshgrid(depth_array, width_array)
    
    for j, thickness in enumerate(df['main_cavity_wall_thickness'].unique()):
        
        select_df = df[(df['mode'] == 'TE011') & (df['main_cavity_wall_thickness'] == thickness)]
        data_array = np.zeros((3, depth_grid.size))
        
        for k, key in enumerate(data_keys):
            
            for n, pair in enumerate(zip(depth_grid.flatten(), width_grid.flatten())):
            
                try:
                    value = select_df[
                        (select_df['groove_depth_fraction'] == pair[0])
                        & (select_df['groove_width'] == pair[1])
                    ][key].item()
                except:
                    pass
                data_array[k, n] = value
                
        te011_data_dict[model_labels[i]][thickness] = data_array
            
        
        

print(te01p_df_list[0]['groove_depth_fraction'].unique())
print(te01p_df_list[0]['groove_width'].unique())

In [ ]:
print(depth_grid, width_grid)

# plot images

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)

In [ ]:
i_parameter_plot = 0 # Q
n_rows = 3

plot_save_path = Path.home()/'group'/'project'/'plots'/\
'analysis'/'hfss'/'cavity'/'220314_1ghz_cavity_optim'
plot_save_path.mkdir(parents=True, exist_ok=True)

for i, model_key in enumerate(te011_data_dict):
    
    for j, thick_key in enumerate(te011_data_dict[model_key]):
        fig = plt.figure(figsize=(13, 8))
        ax = fig.add_subplot(1,1,1)
        img = ax.imshow(
            te011_data_dict[model_key][thick_key][i_parameter_plot, :].reshape(depth_grid.shape) * 1e-0,
            aspect='auto',
            extent = (0.75, 0.25, 2, 12),
            vmin = 60000, 
            vmax = 90000,
            cmap = cmap
        )
        cbar = fig.colorbar(img, label='Q')
        
        ax.set_xlabel('Groove Depth Fraction')
        ax.set_ylabel('Groove Width (mm)')
        ax.set_title(f'Wall Thickness = {thick_key} mm')
        plt.tight_layout()
        name = f'model{model_key}_wallthickness{int(thick_key)}_Qfactor'
        plt.savefig(plot_save_path/name)
        
        plt.show()
    
        


In [ ]:
i_parameter_plot = 2

plot_save_path = Path.home()/'group'/'project'/'plots'/\
'analysis'/'hfss'/'cavity'/'220314_1ghz_cavity_optim'
plot_save_path.mkdir(parents=True, exist_ok=True)

for i, model_key in enumerate(te011_data_dict):
    
    
    for j, thick_key in enumerate(te011_data_dict[model_key]):
        fig = plt.figure(figsize=(13,8))
        
        ax = fig.add_subplot(1, 1,1)
        img = ax.imshow(
            te011_data_dict[model_key][thick_key][i_parameter_plot, :].reshape(depth_grid.shape) * 1e-6,
            aspect='auto',
            extent = (0.75, 0.25, 2, 12),
            cmap = cmap,
            vmin = 0.0, 
            vmax = 5.0,
        )
        cbar = fig.colorbar(img, label=r'$\Delta f$')
        
        ax.set_xlabel('Groove Depth Fraction')
        ax.set_ylabel('Groove Width (mm)')
        ax.set_title(f'Wall Thickness = {thick_key} mm')
        
        plt.tight_layout()
        name = f'model{model_key}_wallthickness{int(thick_key)}_deltaF'
        plt.savefig(plot_save_path/name)
        
        plt.show()
        
        

In [ ]:
i_parameter_plot = 1

for i, model_key in enumerate(te011_data_dict):
    fig = plt.figure(figsize=(32, 6))
    
    for j, thick_key in enumerate(te011_data_dict[model_key]):
        
        ax = fig.add_subplot(1, len((te011_data_dict[model_key].keys())), j+1)
        img = ax.imshow(
            np.log(te011_data_dict[model_key][thick_key][i_parameter_plot, :].reshape(depth_grid.shape) * 1e-9),
            aspect='auto',
            extent = (0.9, 0.1, 1, 16.4)
        )
        cbar = fig.colorbar(img)
        ax.set_title(f'model{model_key}_thickness{thick_key}')
        
    plt.show()
        
        

In [ ]:
plt.imshow(
    te011_data_dict[5][25.0][2, :].reshape(depth_grid.shape) * 1e-3, 
    aspect='auto',
)
plt.colorbar()

In [ ]:
score_params = ['Q', 'min_delta_f', 'groove_depth_fraction', 'main_cavity_wall_thickness']
te011_df = RankTE011Variations(te01p_df, score_params)

In [ ]:
te011_df[0]

In [ ]:
score = 0.25 * (
    + (1 / te011_df['norm_main_cavity_wall_thickness']) ** 2
    + (1 / te011_df['norm_groove_depth_fraction']) ** 2
    + (te011_df['norm_min_delta_f']) ** 2
    + (te011_df['norm_Q']) ** 2
)

In [ ]:
print(score)

In [ ]:
score.idxmax()

In [ ]:
te011_df.loc[score.idxmax()]